In [187]:
import numpy as np
import pandas as pd
import operator
import pprint
import math

In [218]:
def variance(class_data_points):
    if len(list(class_data_points))==0:
        return float(0.0)
    else:    
        return float(np.var(class_data_points))

In [219]:
def information_gain(variance_levelbefore, variance_oflevel, data_points_in_each_node):
    return variance_levelbefore - (np.sum((variance_oflevel*data_points_in_each_node)/np.sum(data_points_in_each_node)))

In [220]:
def find_subtree(dataset,feature,side):
    if side == 'left':
        f= feature.split(" ")
        l= dataset[dataset[f[0]] >= float(f[1])]
        return l
    else:
        f= feature.split(" ")
        l= dataset[dataset[f[0]] < float(f[1])]
        return l

In [221]:
def select_feature(dataset,prev_variance,variance_cache):
    en={}
    info={}
    cols=dataset.columns
    cols=list(cols)
    del cols[-1]
    
    for feature in cols:
        for s in set(list(dataset[feature])):
                left= dataset[dataset[feature] >= s]
                right=dataset[dataset[feature] < s]
                
                left_distribution =np.asarray(list(left['MEDV']))
                
                right_distribution =np.asarray(list(right['MEDV']))
                
                
                if str(left_distribution) not in variance_cache :
                    en[feature+" left"] = variance(left_distribution)
                    variance_cache[str(left_distribution)]=en[feature+" left"]
                else:
                    en[feature+" left"] =variance_cache[str(left_distribution)]

                if str(right_distribution) not in variance_cache:
                    en[feature+" right"] = variance(right_distribution)
                    variance_cache[str(right_distribution)]=en[feature+" right"]
                else:
                    en[feature+" right"]=variance_cache[str(right_distribution)]
                
                info[str(feature)+" "+str(s)]=information_gain(prev_variance,np.asarray([float(en[feature+" left"]),float(en[feature+" right"])]),np.asarray([len(left),len(right)]))
    best_feature=max(info.items(), key=operator.itemgetter(1))[0]
    return best_feature

In [230]:
def training(dataset,depth,tree,variance_cache={}):
    current_node_variance = (variance(np.asarray(list(dataset['MEDV']))))
    print(len(dataset))
    if (depth !=0 and current_node_variance>=0.3):
        
            current_node_variance = variance(np.asarray(list(dataset['MEDV'])))
            best_feature = select_feature(dataset,current_node_variance,variance_cache)
            print(best_feature)
            left_subtree= find_subtree(dataset,best_feature,"left")
            right_subtree=find_subtree(dataset,best_feature,"right")
            if not left_subtree.empty:
                tree["> "+str(best_feature)]=training(left_subtree,depth-1,{},variance_cache)
            if not right_subtree.empty:
                tree["< "+str(best_feature)]=training(right_subtree,depth-1,{},variance_cache)
            return tree
    else:
        print(dataset)
        print("leaf node")
        return np.average(np.asarray(list(dataset['MEDV'])))

In [223]:
training_data = open('housing_train.txt').read().split("\n")
training_data = [i.split(" ") for i in (training_data)]
training_data=[list(filter(lambda j: len(j) > 0, x)) for x in training_data]
training_data=list(filter(lambda x: len(x) > 0, training_data))
column = open('features','r').read().split("\n")
column =[i.strip() for i in column]
training_dataset =  pd.DataFrame(training_data, columns=column)
training_dataset=training_dataset.convert_objects(convert_numeric=True)
training_dataset=training_dataset.dropna()
columnmax = [max(training_dataset[i]) for i in training_dataset.columns]
columnmin = [min(training_dataset[i]) for i in training_dataset.columns]
columns = list(training_dataset.columns)
del columns[-1]
print(columns)
for i,j in enumerate(columns):
    training_dataset[j] = (training_dataset[j] - columnmin[i])/columnmax[i]
# dataset['class']= dataset['class'].astype(int)
# training_dataset=dataset.sample(frac=1)[:-1000]
# test_dataset=dataset.sample(frac=1)[-1000:]

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


/Users/shaivalpatel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [231]:
tree={}
tree=training(training_dataset,12,tree)

433
RM 0.3888382687927107
65
RM 0.44339407744874715
29
PTRATIO 0.2830188679245284
2
CRIM 0.03897626556315059
1
         CRIM   ZN     INDUS  CHAS       NOX        RM  AGE       DIS  \
313  0.038976  0.0  0.687719   1.0  0.374282  0.594419  0.8  0.063918   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
313  0.958333  0.719219  0.358491  0.892492  0.096268  21.9  
leaf node
1
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
231  0.002423  0.2  0.253411   1.0  0.082664  0.470387  0.489  0.266928   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
231  0.083333  0.054054  0.283019  0.983749  0.131152  35.2  
leaf node
27
B 0.9958175862937768
2
CRIM 0.0012869733704069177
1
        CRIM   ZN     INDUS  CHAS      NOX       RM    AGE       DIS  \
84  0.001287  0.0  0.094737   0.0  0.06085  0.51344  0.731  0.195077   

         RAD       TAX   PTRATIO         B     LSTAT  MEDV  
84  0.041667  0.133634  0.254717  0.999194  0.067063  38.7  

TAX 0.12462462462462462
15
AGE 0.834
1
        CRIM   ZN    INDUS  CHAS       NOX        RM    AGE       DIS  \
75  0.000565  0.0  0.11501   0.0  0.111366  0.392483  0.834  0.189016   

         RAD       TAX   PTRATIO         B     LSTAT  MEDV  
75  0.041667  0.124625  0.245283  0.999194  0.101947  23.6  
leaf node
14
CRIM 0.00045337966782128255
8
CRIM 0.0006999624618718263
5
CRIM 0.004563692313225335
3
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
192  0.004564  0.0  0.223782   0.0  0.128588  0.410251  0.770  0.172028   
196  0.005132  0.0  0.223782   0.0  0.128588  0.438610  0.740  0.209615   
202  0.005681  0.0  0.223782   0.0  0.132032  0.432460  0.687  0.248909   

          RAD      TAX   PTRATIO         B     LSTAT  MEDV  
192  0.291667  0.18018  0.226415  0.936659  0.125203  31.6  
196  0.291667  0.18018  0.226415  0.946888  0.095187  31.7  
202  0.291667  0.18018  0.226415  0.981985  0.081125  31.5  
leaf node
2
         CRIM    ZN     INDUS  CHAS

LSTAT 0.7387777176852354
3
CRIM 0.20318107538869942
2
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
356  0.514068  0.0  0.687719   0.0  0.345580  0.109112  0.971  0.043590   
357  0.203181  0.0  0.687719   0.0  0.329506  0.327221  0.971  0.058145   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
356  0.958333  0.719219  0.358491  0.221592  0.953218   7.0  
357  0.958333  0.719219  0.358491  0.067851  0.738778   7.2  
leaf node
1
         CRIM   ZN     INDUS  CHAS       NOX        RM   AGE       DIS  \
377  0.153657  0.0  0.687719   0.0  0.399541  0.270387  0.85  0.056983   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
377  0.958333  0.719219  0.358491  0.172915  0.873175   8.4  
leaf node
5
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
360  0.826376  0.0  0.687719   0.0  0.329506  0.272893  0.971  0.055498   
361  0.132687  0.0  0.687719   0.0  0.374282  0.371640  0.736  0.054789   
367  0.17818

CRIM 0.12477246724404951
1
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
322  0.124772  0.0  0.687719   0.0  0.316877  0.153189  0.971  0.003678   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
322  0.958333  0.719219  0.358491  0.999194  0.893456  13.8  
leaf node
1
         CRIM   ZN     INDUS  CHAS       NOX        RM   AGE       DIS  \
355  0.083141  0.0  0.687719   0.0  0.235362  0.234169  0.95  0.026809   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
355  0.958333  0.719219  0.358491  0.791938  0.667117  17.2  
leaf node
19
CRIM 0.10685228184615661
6
NOX 0.37428243398392647
3
RM 0.3330296127562643
1
         CRIM   ZN     INDUS  CHAS       NOX       RM    AGE       DIS  \
382  0.111943  0.0  0.687719   0.0  0.399541  0.33303  0.971  0.069995   

          RAD       TAX   PTRATIO        B     LSTAT  MEDV  
382  0.958333  0.719219  0.358491  0.97357  0.462953  15.4  
leaf node
2
         CRIM   ZN     INDUS  CHAS      

DIS 0.07229621077804808
28
AGE 0.551
24
RM 0.32072892938496583
1
        CRIM     ZN     INDUS  CHAS      NOX        RM    AGE       DIS  \
10  0.002456  0.125  0.288889   0.0  0.15155  0.320729  0.914  0.430223   

         RAD       TAX   PTRATIO         B    LSTAT  MEDV  
10  0.166667  0.186186  0.122642  0.988158  0.50622  15.0  
leaf node
23
RM 0.29738041002277904
2
CRIM 0.0028918969342363464
1
        CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
93  0.002892  0.0  0.315789   0.0  0.146958  0.303872  0.883  0.116728   

         RAD       TAX   PTRATIO         B     LSTAT  MEDV  
93  0.166667  0.295796  0.391509  0.984908  0.373716  19.4  
leaf node
1
       CRIM     ZN     INDUS  CHAS      NOX       RM    AGE       DIS  \
7  0.001554  0.125  0.288889   0.0  0.15155  0.29738  0.932  0.397551   

        RAD       TAX   PTRATIO         B     LSTAT  MEDV  
7  0.166667  0.186186  0.122642  0.999194  0.471065  27.1  
leaf node
21
INDUS 0.3949317738791423
8
DIS 0.2

CRIM 0.001350698276617792
1
        CRIM    ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
55  0.001351  0.25  0.182066   0.0  0.070034  0.364579  0.405  0.564986   

         RAD       TAX   PTRATIO         B     LSTAT  MEDV  
55  0.291667  0.145646  0.334906  0.995868  0.210114  25.0  
leaf node
1
         CRIM    ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
298  0.000215  0.55  0.129435   0.0  0.105626  0.357062  0.535  0.379541   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
298  0.166667  0.274775  0.235849  0.999194  0.147377  23.9  
leaf node
2
        CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
82  0.001301  0.0  0.094737   0.0  0.060850  0.348975  0.549  0.195077   
87  0.001600  0.0  0.315789   0.0  0.146958  0.360592  0.770  0.135917   

         RAD       TAX   PTRATIO         B     LSTAT  MEDV  
82  0.041667  0.133634  0.254717  0.901134  0.133045  28.4  
87  0.166667  0.295796  0.391509  0.993802  0.207950 

LSTAT 0.1587344510546241
144
TAX 0.05405405405405406
138
PTRATIO 0.39622641509433965
14
LSTAT 0.17658193618171986
13
CRIM 0.01515461438002522
1
        CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS    RAD  \
26  0.015155  0.0  0.299415   0.0  0.167623  0.285991  0.971  0.251136  0.125   

        TAX   PTRATIO         B     LSTAT  MEDV  
26  0.18018  0.396226  0.948375  0.305841  14.5  
leaf node
12
LSTAT 0.3477555435370471
3
CRIM 0.003888792733337679
2
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
17   0.008743  0.0  0.299415   0.0  0.167623  0.276651  0.788  0.257972   
116  0.003889  0.0  0.835478   0.0  0.266361  0.329499  0.955  0.059391   

       RAD       TAX   PTRATIO         B     LSTAT  MEDV  
17   0.125  0.180180  0.396226  0.973621  0.349919  17.5  
116  0.125  0.375375  0.405660  0.992089  0.347756  17.1  
leaf node
1
        CRIM    ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
46  0.000082  0.75  0.138012   0.0 

PTRATIO 0.2830188679245284
6
         CRIM    ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
174  0.002510  0.00  0.394932   0.0  0.111366  0.314920  0.496  0.265971   
176  0.001456  0.00  0.394932   1.0  0.111366  0.285080  0.562  0.256430   
183  0.002155  0.00  0.394932   0.0  0.111366  0.298519  0.395  0.232202   
213  0.001777  0.22  0.210526   0.0  0.044776  0.327107  0.462  0.552253   
230  0.001217  0.20  0.253411   0.0  0.082664  0.339066  0.558  0.229901   
282  0.002638  0.00  0.269786   0.0  0.115959  0.287244  0.408  0.353466   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
174  0.125000  0.135135  0.283019  0.994079  0.249865  24.4  
176  0.125000  0.135135  0.283019  0.959940  0.349648  24.4  
183  0.125000  0.135135  0.283019  0.990955  0.209302  25.0  
213  0.250000  0.214715  0.306604  0.943285  0.210654  24.5  
230  0.083333  0.054054  0.283019  0.994306  0.162250  24.4  
282  0.166667  0.150150  0.330189  0.999194  0.299081  22.2  
leaf

CHAS 1.0
2
CRIM 0.04738424432601077
1
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
312  0.047384  0.0  0.687719   1.0  0.433984  0.255353  0.861  0.063918   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
312  0.958333  0.719219  0.358491  0.888687  0.349108  16.8  
leaf node
1
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
129  0.012591  0.0  0.745419   1.0  0.549943  0.165262  0.851  0.039632   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
129  0.166667  0.324324  0.099057  0.864097  0.280963  15.3  
leaf node
21
RAD 0.16666666666666666
14
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
31   0.001024  0.0  0.214425   0.0  0.122847  0.259681  0.585  0.185404   
103  0.001556  0.0  0.372320   0.0  0.177956  0.247153  0.623  0.134383   
128  0.016746  0.0  0.745419   0.0  0.549943  0.209909  0.971  0.038098   
145  0.001493  0.0  0.139961   0.0  0.135476  0.229043  0.85

B 0.91708238851096
6
CRIM 0.008812356562766223
2
CRIM 0.015944713305355813
1
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
136  0.015945  0.0  0.745419   0.0  0.549943  0.335877  0.971  0.052472   

          RAD       TAX   PTRATIO         B     LSTAT  MEDV  
136  0.166667  0.324324  0.099057  0.917082  0.153056  23.3  
leaf node
1
         CRIM   ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
266  0.008812  0.0  0.368031   0.0  0.174512  0.291686  0.499  0.124578   

       RAD       TAX   PTRATIO         B     LSTAT  MEDV  
266  0.125  0.175676  0.273585  0.999194  0.114927  22.1  
leaf node
4
         CRIM    ZN     INDUS  CHAS       NOX        RM    AGE       DIS  \
66   0.000816  0.00  0.482261   0.0  0.051665  0.308884  0.031  0.257444   
80   0.000252  0.28  0.568421   0.0  0.082664  0.301822  0.260  0.209154   
150  0.000539  0.00  0.139961   0.0  0.135476  0.313667  0.705  0.180423   
229  0.001751  0.20  0.253411   0.0  0.082664  0.30

In [226]:
def testing(row,tree,depth):
 
    if  depth !=0 and type(tree)==dict :
        for key in tree.keys():
            operator = key.split(" ")[0]
            feature =key.split(" ")[1]
            threshold=key.split(" ")[2]
            row_feature = row[feature]
            if operator == '<':
                if (row_feature)< float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
            else:
                if (row_feature)>= float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
    else:
        
        return float(tree)

In [233]:
prediction=[]
for index,row in testing_dataset.iterrows():
    prediction.append(testing(row,tree,12))

In [225]:
testing_data = open('housing_test.txt').read().split("\n")
testing_data = [i.split(" ") for i in (testing_data)]
testing_data=[list(filter(lambda j: len(j) > 0, x)) for x in testing_data]
testing_data=list(filter(lambda x: len(x) > 0, testing_data))
column = open('features','r').read().split("\n")
column =[i.strip() for i in column]
testing_dataset =  pd.DataFrame(testing_data, columns=column)
testing_dataset=testing_dataset.convert_objects(convert_numeric=True)
columnmax = [max(testing_dataset[i]) for i in testing_dataset.columns]
columnmin = [min(testing_dataset[i]) for i in testing_dataset.columns]
columns = list(testing_dataset.columns)
del columns[-1]
print(columns)
for i,j in enumerate(columns):
    testing_dataset[j] = (testing_dataset[j] - columnmin[i])/columnmax[i]
# dataset['class']= dataset['class'].astype(int)
# testing_dataset=dataset.sample(frac=1)[:-1000]
# test_dataset=dataset.sample(frac=1)[-1000:]

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


/Users/shaivalpatel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [236]:
def accuracy(prediction,testing_dataset):
    test=np.asarray(list(testing_dataset['MEDV']))
    pred =np.asarray(list(prediction))
    
    return np.sum((test-pred)**2)/len(pred)

In [237]:
accuracy(prediction,testing_dataset)

25.451517857525896

In [232]:
tree

{'> RM 0.3888382687927107': {'> RM 0.44339407744874715': {'> PTRATIO 0.2830188679245284': {'> CRIM 0.03897626556315059': 21.9,
    '< CRIM 0.03897626556315059': 35.2},
   '< PTRATIO 0.2830188679245284': {'> B 0.9958175862937768': {'> CRIM 0.0012869733704069177': 38.7,
     '< CRIM 0.0012869733704069177': 39.8},
    '< B 0.9958175862937768': {'> CRIM 0.0064289102029531486': 50.0,
     '< CRIM 0.0064289102029531486': {'> TAX 0.11561561561561562': {'> LSTAT 0.05922120064899946': {'> RM 0.5509111617312074': {'> CRIM 0.005851115242053493': 50.0,
         '< CRIM 0.005851115242053493': 48.8},
        '< RM 0.5509111617312074': {'> AGE 0.7539999999999999': {'> CRIM 0.005931923368271514': 43.1,
          '< CRIM 0.005931923368271514': 44.8},
         '< AGE 0.7539999999999999': {'> CRIM 0.0036543480166606346': 48.3,
          '< CRIM 0.0036543480166606346': 46.7}}},
       '< LSTAT 0.05922120064899946': {'> AGE 0.836': 37.6,
        '< AGE 0.836': {'> DIS 0.22337855110707955': 42.0,
         '